In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os

In [ ]:
import glob

In [ ]:
os.environ["SM_FRAMEWORK"] = "tf.keras"

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
!pip install -U -q segmentation-models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 KB 2.4 MB/s eta 0:00:00


In [ ]:
import segmentation_models as sm

Segmentation Models: using `tf.keras` framework.


In [ ]:
def prepare_dataset_2(path_X,path_y,shape):
  images = []
  for directory_path in glob.glob(path_X):
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path)       
        img = cv2.resize(img, (512, 256))
        images.append(img)
       
  images = np.array(images)

  masks = [] 
  for root,dirs,files in os.walk(path_y):
      for file in files:
          mask_path=os.path.join(path_y,file)
          mask = cv2.imread(mask_path)       
          mask = cv2.resize(mask, (512,256))
          train_mask=process_mask_img(mask,3)
          masks.append(train_mask)
        
  masks = np.array(masks)
  return images,masks
  

In [ ]:
def prepare_dataset(path_X,path_y,shape):
  n_classes=3
  width,height=shape

  index=0
  if os.path.exists(path_X) and os.path.exists(path_y):
    
    for root,dis,files in os.walk(path_X):
      X=np.zeros((len(files),height,width,3),dtype=np.uint8)
      y=np.zeros((len(files),height,width,n_classes),dtype=np.uint8)
      for file in files:
        file_num=file.split(".")[0]
        mask_name=file_num+"_mask.png"
        mask_path=os.path.join(path_y,mask_name)
        if os.path.exists(mask_path):
          image=cv2.imread(os.path.join(path_X,file))
          mask_img=cv2.imread(mask_path)
          image=cv2.resize(image,dsize=shape)
          mask_img=cv2.resize(mask_img,dsize=shape)
          mask=process_mask_img(mask_img,n_classes)

          X[index]=image.astype(np.uint8)
          y[index]=mask.astype(np.uint8)
        index+=1
    return X,y



In [ ]:
unique_colors = [[0,0,0],[0,176,0],[0,0,255]]  # considering the bgr coloring
color_map = {tuple(color): i for i, color in enumerate(unique_colors)}

def process_mask_img(img,n_classes):
  labels = np.zeros((img.shape[0], img.shape[1]), dtype=int)
  for color, label in color_map.items():
      mask = np.all(img == color, axis=-1)
      labels[mask] = label
  # plt.imshow(labels,cmap='jet')
  
  train_mask = np.zeros((img.shape[0], img.shape[1], n_classes))
  for c in range(n_classes):
    train_mask[:, :, c] = (labels == c).astype(int)
  return train_mask
  
  

In [ ]:
def create_model(back_bone,n_classes,activation,input_shape):
  model = sm.Unet(back_bone, input_shape=input_shape,encoder_weights='imagenet',classes=n_classes, activation=activation)
  model.summary()
  return model

In [ ]:
model=create_model("resnet34",3,"softmax",(256,512,3))


85521592/85521592 [==============================] - 2s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, 256, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 bn_data (BatchNormalization)   (None, 256, 512, 3)  9           ['data[0][0]']                   
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 262, 518, 3)  0          ['bn_data[0][0]']                
                                                                                                  
 conv0 (Conv2D)            

In [ ]:
train=prepare_dataset_2("/content/drive/MyDrive/images/train","/content/drive/MyDrive/annotations/train",(512,256))
test=prepare_dataset_2("/content/drive/MyDrive/images/test","/content/drive/MyDrive/annotations/test",(512,256))
val=prepare_dataset_2("/content/drive/MyDrive/images/val","/content/drive/MyDrive/annotations/val",(512,256))

In [ ]:
dice_loss = sm.losses.DiceLoss() 
focal_loss = sm.losses.CategoricalFocalLoss()
TOTAL_LOSS = dice_loss + (1 * focal_loss)
METRICS = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]
model.compile(optimizer="adam",loss=TOTAL_LOSS,metrics=METRICS)

In [ ]:
preprocess_input1 = sm.get_preprocessing("resnet34")

# preprocess input
train = preprocess_input1(train)
test = preprocess_input1(test)

In [ ]:
model.fit(train[0],train[1],epochs=10,validation_data=(test[0],test[1]),batch_size=8)

Epoch 1/10
50/50 [==============================] - 25s 281ms/step - loss: 0.4138 - iou_score: 0.5512 - f1-score: 0.6024 - val_loss: 0.4595 - val_iou_score: 0.5157 - val_f1-score: 0.5802
Epoch 2/10
50/50 [==============================] - 14s 282ms/step - loss: 0.4134 - iou_score: 0.5505 - f1-score: 0.6020 - val_loss: 1.0680 - val_iou_score: 0.1799 - val_f1-score: 0.2760
Epoch 3/10
50/50 [==============================] - 14s 280ms/step - loss: 0.4155 - iou_score: 0.5497 - f1-score: 0.6050 - val_loss: 1.3734 - val_iou_score: 0.1646 - val_f1-score: 0.2550
Epoch 4/10
50/50 [==============================] - 14s 284ms/step - loss: 0.4137 - iou_score: 0.5535 - f1-score: 0.6077 - val_loss: 1.6344 - val_iou_score: 0.1205 - val_f1-score: 0.1849
Epoch 5/10
50/50 [==============================] - 14s 286ms/step - loss: 0.4140 - iou_score: 0.5521 - f1-score: 0.6094 - val_loss: 2.2006 - val_iou_score: 0.0069 - val_f1-score: 0.0135
Epoch 6/10
49/50 [============================>.] - ETA: 0s - los

KeyboardInterrupt: ignored